In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [3]:
%%capture
!conda install -c conda-forge openmm mdtraj parmed py3dmol
#install openabc for single bead per aminoacid CG models
#https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1011442
#https://github.com/ZhangGroup-MITChemistry/OpenABC
!pip install openabc

In [2]:
# allow widgets in colab
from google.colab import output
output.enable_custom_widget_manager()
#check openmm installation
#!python -m openmm.testInstallation

In [5]:
# load packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mdtraj
import py3Dmol
import sys
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit

sys.path.append('../../')

# load HPS model with OpenABC
from openabc.forcefields.parsers import HPSParser
from openabc.forcefields import HPSModel
from openabc.utils.helper_functions import build_straight_CA_chain, write_pdb, compute_rg
from openabc.utils.insert import insert_molecules
from openabc.lib import *

In [6]:
# set simulation platform
platform_name = 'CPU'   # CPU or CUDA

In [34]:
ca_pdb = 'cdc15.pdb'
HPSParser.from_atomistic_pdb('cdc15_AF3.pdb',ca_pdb)

protein_parser = HPSParser(ca_pdb)
protein = HPSModel()
protein.append_mol(protein_parser)
top = app.PDBFile(ca_pdb).getTopology()
init_coord = app.PDBFile(ca_pdb).getPositions()

view = py3Dmol.view(width=400, height=400)
view.addModelsAsFrames(open('cdc15.pdb', 'r').read(),'pdb')
view.setBackgroundColor('white')
view.setStyle({'sphere': {'colorscheme':'amino'}})  # amino acid color
view.zoomTo()
view.show()

Parse molecule with default settings.
Parse molecule with default settings.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [35]:
protein.create_system(top, box_a=200, box_b=200, box_c=200)
#rigid bodies
FBARA = np.arange(35, 300)
FBARB = np.arange(949, 1225)
SH3A = np.arange(870,927)
SH3B = np.arange(1797,1854)

rigid_coord = init_coord # set rigid body coordinates
rigid_bodies = [FBARA.tolist()+FBARB.tolist(),SH3A.tolist(),SH3B.tolist()]
protein.set_rigid_bodies(rigid_coord, rigid_bodies)

protein.add_protein_bonds(force_group=1)
#KR HPS scale as in original Dignon et al. HPS paper.
protein.add_contacts('KR', mu=1, delta=0, force_group=2)
#Used in OpenABC example: protein.add_contacts('Urry', mu=1, delta=0.08, force_group=2)
#Default Debye screening length = 1 nm (~ 100 mM ionic strength) ; dielectric constant = 80
protein.add_dh_elec(force_group=3)
temperature = 300*unit.kelvin
friction_coeff = 1/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.LangevinIntegrator(temperature, friction_coeff, timestep)
protein.set_simulation(integrator, platform_name, init_coord=init_coord)

Add protein bonds.
Add nonbonded contacts.
Use KR hydropathy scale.
Scale factor mu = 1 and shift delta = 0.
Add Debye-Huckel electrostatic interactions.
Set Debye length as 1 nm.
Set water dielectric as 80.0.
Use platform: CPU


In [36]:
#%%capture
#protein.simulation.minimizeEnergy()
output_interval = 1000 # measused in number of timesteps: 1000 steps = 10 ps for timestep = 10 fs
nanosec = 100000 # measured in number of timesteps: 100000 steps = 1 ns for timestep = 10 fs
output_dcd = 'output.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)
protein.simulation.step(int(0.05*nanosec))

#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
1000,9.999999999999831,406815.12393202115,4634.1466626048705,411449.270594626,308.6155773670415,0
2000,20.000000000000327,406847.6760440259,4428.852519438106,411276.528563464,294.9438109909921,62.7
3000,30.00000000000189,406996.1930387575,4257.4444046761255,411253.63744343363,283.5287181693657,66.4
4000,40.00000000000061,406875.3711584415,4403.019352713329,411278.39051115484,293.2234257195633,68.5
5000,49.99999999999862,406893.3737472764,4346.366621268581,411239.74036854494,289.45058107368135,70.2


In [38]:
# visualize specific frame of trajectory
#frame_number = 2
traj = mdtraj.load_dcd('output.dcd', top=ca_pdb)
#frame = traj[frame_number]
frame = traj  # to visualize whole trajectorie as movie
frame.save_pdb('frame.pdb')
view = py3Dmol.view(width=400, height=400)
view.addModelsAsFrames(open('frame.pdb', 'r').read(),'pdb')
view.setBackgroundColor('white')

# Colorschemes https://3dmol.csb.pitt.edu/doc/global.html#builtinColorSchemes
#view.setStyle({'sphere': {'color':'green'}})  # same color
#view.setStyle({'sphere':{'colorscheme':{'prop':'resi','gradient':'roygb','min':26,'max':109}}})  # gradient color
view.setStyle({'sphere': {'colorscheme':'amino'}})  # amino acid color
#view.setStyle({'sphere': {'colorscheme':'shapely'}})  # amino acid color
view.zoomTo()
#view.zoomTo({'model':-1})
view.animate({'loop': "forward", 'interval': 1000})  # for movies
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.